In [1]:
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [6]:
class OffensiveWordsDetectorWithContext:
    def __init__(self):
        # רשימת המילים הפוגעניות
        self.offensive_words = [
        "אהצבועה", "אובססיבית", "איכס", "אינעל","שמוק","דביל","בולבול","קפוץ לי", "אנטיפתי", "אסתור", "ב\"ביצים", "בביצים", "בגועליציה", "בגזענותו",
        "בוזין", "בזוי", "בזויים", "בחרא", "ביריונים", "בנזונה", "בקקה", "בשמוק", "בתחת", "בתחתונים", "גונב", "גונבי",
        "גנב", "גנבים", "גנבת", "דימגוג", "דפוק", "דפוקים", "דרקים", "הארכיטיפ", "הבוגדים", "הזבל", "הזבלל", "הזדוניים",
        "הזוי", "הזויה", "הזויים", "החאפרים", "החארות", "החולנית", "החולרה", "החוצפן", "החמור", "יחמור", "החפרן",
        "היללן", "המבהיל", "המבחילה", "המושרצים", "המזורגג", "המחורבן", "המחורבנות", "המחורבנים", "המחורבנת", "המטומטם",
        "המטומטמת", "המנוולת", "המניאקים", "המנייאק", "המסריח", "המסריחים", "המפלצת", "המתוסבכתנכדתו", "הסטן",
        "הסרחוני", "הפאקינג", "הפושע", "הפושעים", "הפראייר", "הציבוז'י", "הקקה", "הרשע", "הרשעים", "הרשעית", "השמוק",
        "השמוקים", "השקרן", "השקרנית", "השרלטנות", "התחת", "וגנב", "ודרעק", "והנבער", "והנצלנית", "והשפל", "וחלאות",
        "וחרא", "וכוסאמק", "ולברבר", "ולעלובי", "ומנוול", "ומנוולים", "ומסריח", "וסורר", "וסטנים", "וסמרטוטית",
        "וערמומית", "ופחדן", "ופטפטני", "ופסיכופטים", "וצבוע", "וצבועה", "וציפורע", "וקשקשן","קשקשן" "ורמאי", "ורמאים",
        "ושקרן", "ושקרנים", "ושקרנית", "זבל", "זבלה", "זבלים", "זומבים", "חאלס", "חאפר", "חאפרים", "חאפרית",
        "חובבן", "חולני", "חזיר", "חירבנה", "חמוריםםם", "חרא", "חראא", "חראאא", "טינופת", "טמטום", "יאוכלי", "יבן",
        "יוק", "יזבל", "יזבלה", "יזבל", "יחתיכת", "יטמבל", "יליצן", "ימגעיל", "ימיץ", "ימניאק", "ימניייק", "ימנייק",
        "ימניק", "ימעפן", "ינעל", "כאןצבועה", "ככלבלב", "יכלב", "כנפיחה", "כפודל", "כשחלאה", "לאזעזל", "להזויים",
        "להשתין", "לזרגג", "לחאפר", "לחרא", "לחרבן", "לךגזען", "למותי", "לעזאזאל", "לעזזאל", "לקק", "לשקרנים",
        "מאוסה", "מאפן", "מבהיל", "מבהילה", "מבהילים", "מבחילה", "מגעיל", "מגעילה", "מגעילים", "מהאדיוט", "מזוויעה",
        "מזוויעים", "מזורגג", "מזנים", "מחורבן", "מחורבנות", "מחורבנת", "מטונף", "מלהתקרצץ", "מלשין", "מלשינה",
        "ממזרים", "מנובל", "מנובלים", "מניאק", "מנייאק", "מנייק", "מניפולטורית", "מניפולטיבית", "מניק", "מנניאיק",
        "מסקן", "מסקנים", "מסריח", "מסריחה", "מסריחות", "מסריחים", "מעפן", "מעפנהה", "מפגר", "מפגרים", "מפגרת",
        "מפחידה", "משקר", "משקרים", "משתין", "משתינה", "משתינות", "משתינים", "נבזי", "נבזים", "נוכל", "נוכלות",
        "נוכלים", "נוכלת", "נחש", "ניבזה", "נצלנים", "סאדיסטי", "סטן", "סטנים", "סטרפלצט", "סמרטוט", "עוכר",
        "עוכרי", "עוכרת", "פָּתט", "פאקינג", "פברק", "פברקו", "פושע", "פושעת", "יפח", "פחדנים", "פחזבל", "פיפי",
        "פלוץ", "פתטי", "פתי", "צבוע", "צבועה", "צבועים", "קברן", "קברניות", "קברנים", "קברנית", "קיבינמט",
        "קקאיות", "קקה", "קקות", "קקי", "קרימינל", "קרימינלית", "רבאק", "רמאי", "רמאים", "רמאית", "רשלן",
        "שאובססיבית", "שהמפלצת", "שודד", "שונא", "שונאת", "שטחית", "שלוזרים", "שמוק", "שמלאני", "שמלאנים",
        "שמלנים", "שפיצרקה", "שקלוזה", "שקרן", "שקרני", "שקרניות", "שקרנים", "שקרנית", "שרלטן", "שרלטנים",
        "שרלטנית", "שתחנק", "שתמות", "שתמותי", "שתסתמי", "שתשרפי", "תחמן", "תחמניות", "תחמנים", "תחמנית",
        "תיסתום", "תישארמה", "תמותי", "תנענע", "תסתום", "תסתמו", "תסתמי", "תעופי", "תעפו", "תשרף", "תשתכשכי"
        ]
        
        # ביטויים של קריאה לסדר
        self.order_calls = [
            'אני קורא אותך לסדר',
            'אני קוראת אותך לסדר',
            'זאת אזהרה אחרונה',
            'קריאה לסדר'
        ]
        
        # סימוני הפרעה
        self.interruption_marks = ["- - -", "- -"]
    def analyze_text(self, text):
        """Analyze a single text for all types of content"""
        if not isinstance(text, str):
            return {
                'offensive_found': 0,
                'order_calls_found': 0,
                'interruption_found': 0,
                'found_words': ''
            }
        
        text_lower = text.lower()
        found_words = []
        
        # Check for offensive words
        offensive_found = 0
        for word in self.offensive_words:
            if word in text_lower:
                offensive_found = 1
                found_words.append(word)
        
        # Check for order calls
        order_calls_found = 0
        for call in self.order_calls:
            if call in text:
                order_calls_found = 1
                found_words.append(call)
        
        # Check for interruptions
        interruption_found = 0
        for mark in self.interruption_marks:
            if mark in text:
                interruption_found = 1
                found_words.append(mark)
        
        return {
            'offensive_found': offensive_found,
            'order_calls_found': order_calls_found,
            'interruption_found': interruption_found,
            'found_words': ', '.join(found_words) if found_words else ''
        }

    def process_batch(self, batch_num):
        """Process a single batch file"""
        try:
            file_path = f"/home/gorelikk/NLP-PROJECT/Pre_Process/Pre_Process/batch_{batch_num}_processed_data.csv"
            df = pd.read_csv(file_path)
            print(f"\nProcessing batch {batch_num}, {len(df)} rows")
            
            # Initialize lists for columns
            offensive_flags = []
            order_call_flags = []
            interruption_flags = []
            found_words_list = []
            
            # Process each row
            for _, row in tqdm(df.iterrows(), total=len(df)):
                analysis = self.analyze_text(row['conversation'])
                offensive_flags.append(analysis['offensive_found'])
                order_call_flags.append(analysis['order_calls_found'])
                interruption_flags.append(analysis['interruption_found'])
                found_words_list.append(analysis['found_words'])
            
            # Add new columns to the dataframe
            df['offensive_words_found'] = offensive_flags
            df['order_calls_found'] = order_call_flags
            df['interruption_found'] = interruption_flags
            df['found_words'] = found_words_list
            
            return df
            
        except Exception as e:
            print(f"Error processing batch {batch_num}: {e}")
            return None

    def process_all_batches(self):
        """Process all batch files"""
        all_results = []
        
        for batch_num in range(1, 16):
            batch_df = self.process_batch(batch_num)
            if batch_df is not None:
                all_results.append(batch_df)
        
        if all_results:
            # Combine all results
            final_df = pd.concat(all_results, ignore_index=True)
            
            # Save the results
            final_df.to_csv("Detect/all_batches_analysis.csv", index=False)
            print(f"\nResults saved to: all_batches_analysis.csv")
            
            # Print summary statistics
            total_rows = len(final_df)
            offensive_count = sum(final_df['offensive_words_found'])
            order_calls_count = sum(final_df['order_calls_found'])
            interruption_count = sum(final_df['interruption_found'])
            
            print("\n=== Summary ===")
            print(f"Total conversations analyzed: {total_rows:,}")
            print(f"Conversations with offensive words: {offensive_count:,} ({offensive_count/total_rows*100:.2f}%)")
            print(f"Conversations with order calls: {order_calls_count:,} ({order_calls_count/total_rows*100:.2f}%)")
            print(f"Conversations with interruptions: {interruption_count:,} ({interruption_count/total_rows*100:.2f}%)")
            
            return final_df
        return None

In [ ]:
if __name__ == "__main__":
    detector = OffensiveWordsDetectorWithContext()
    results = detector.process_all_batches()


Processing batch 1, 509958 rows


100%|██████████| 509958/509958 [00:37<00:00, 13452.48it/s]



Processing batch 2, 538803 rows


100%|██████████| 538803/538803 [00:39<00:00, 13783.19it/s]



Processing batch 3, 480954 rows


...

100%|██████████| 556595/556595 [00:36<00:00, 15351.34it/s]



Processing batch 9, 961182 rows


100%|██████████| 961182/961182 [01:01<00:00, 15711.47it/s]



Processing batch 10, 688563 rows


 43%|████▎     | 295858/688563 [00:19<00:26, 15020.42it/s]